In [1]:
import numpy as np

In [2]:
np.random.seed(0)

In [3]:
data = "hello world"
chars = list(set(data))
vocab_size = len(chars)

chars_to_indices = {char: i for i, char in enumerate(chars)}
indices_to_chars = {i: char for i, char in enumerate(chars)}

chars_to_indices, indices_to_chars

({'w': 0, 'o': 1, ' ': 2, 'l': 3, 'h': 4, 'r': 5, 'e': 6, 'd': 7},
 {0: 'w', 1: 'o', 2: ' ', 3: 'l', 4: 'h', 5: 'r', 6: 'e', 7: 'd'})

In [4]:
# RNN configs
embedding_size = vocab_size
hidden_size = 64
output_size = vocab_size

In [5]:
# Matrices
Wxh = np.random.randn(hidden_size, embedding_size)
Whh = np.random.randn(hidden_size, hidden_size)
bh = np.random.randn(1, hidden_size)

Why = np.random.randn(output_size, hidden_size)
by = np.random.randn(1, output_size)

In [6]:
def softmax(z):
    e_z = np.exp(z)
    return e_z / e_z.sum(axis=1)

def rnn(x_t, h_prev):
    assert x_t.shape == (1, embedding_size)
    assert h_prev.shape == (1, hidden_size)

    h_next = np.tanh(x_t @ Wxh.T + h_prev @ Whh.T + bh)
    probs = softmax(h_next @ Why.T + by)
    
    return probs, h_next

def sample(char, n):
    # sample rnn n times starting with the first char
    x = np.zeros((1, vocab_size))
    idx = chars_to_indices[char]
    x[:, idx] = 1 # one hot encoding
    h =  np.zeros((1, hidden_size)) # hidden state
    idxes = []
    
    for i in range(n):
        probs, h = rnn(x, h) # xt, hprev -> rnn -> probs, hnext
        idx = np.random.choice(vocab_size, p=probs.ravel()) # .ravel returns a 1d array
        
        x = np.zeros((1, vocab_size))
        x[:, idx] = 1
        idxes.append(idx)

    chars = "".join([indices_to_chars[i] for i in idxes])
    return chars

In [7]:
x_t = np.random.randn(1, embedding_size)
h_prev = np.random.randn(1, hidden_size)

probs, h_next = rnn(x_t, h_prev)
probs.shape, h_next.shape

((1, 8), (1, 64))

In [8]:
inp = [chars_to_indices[c] for c in data[:-1]]
target = [chars_to_indices[c] for c in data[1:]]

inp, target

([4, 6, 3, 3, 1, 2, 0, 1, 5, 3], [6, 3, 3, 1, 2, 0, 1, 5, 3, 7])

In [9]:
sample("h", 4)

'drre'

In [14]:
lr = 1e-2
for e in range(100):
    # compute loss
    loss = 0
    xs, ps, hs = {}, {}, {}
    h = np.zeros((1, hidden_size))
    for t in range(len(inp)):
        idx = inp[t]
        x = np.zeros((1, vocab_size))
        x[:, idx] = 1
        probs, h = rnn(x, h)
        xs[t] = x
        ps[t] = probs # save probs (we'll use this in backprop)
        hs[t] = h

        # cross entropy
        pred = probs[0, target[t]]
        loss += -np.log(pred)

    # backprop, calculate gradients
    dL_dWxh = np.zeros_like(Wxh)
    dL_dWhh = np.zeros_like(Whh)
    dL_dbh = np.zeros_like(bh)
    dL_Why = np.zeros_like(Why)
    dL_dby = np.zeros_like(by)
    dF_dh = np.zeros((1, hidden_size))

    for t in reversed(range(len(inp))):
        dL_dz2 = np.copy(ps[t])
        target_idx = target[t]
        dL_dz2[:, target_idx] -= 1

        # print(dL_dz2.shape)

        a = hs[t] * dL_dz2.T
        # print(a.shape)

        # 2nd layer
        dL_Why += hs[t] * dL_dz2.T
        dL_dby += dL_dz2


        # 1st layer
        dh_dz1 = 1 - hs[t] ** 2
        dL_dh = dF_dh + (dL_dz2 @ Why) 

        # print((xs[t] * dh_dz1.T * dL_dh.T).shape)

        dL_dWxh += xs[t] * dh_dz1.T * dL_dh.T
        dL_dWhh +=  hs[t] * dh_dz1.T * dL_dh.T
        dL_dbh +=  dh_dz1 * dL_dh

        # raise RuntimeError

        # dF_dh = dL_dh * (dh_dz1 @ Whh)

    # clip gradients
    for gradient in [dL_dWxh, dL_dWhh, dL_Why, dL_dbh, dL_dby]:
        np.clip(gradient, -1, 1, out=gradient)
        # np.clip(gradient, -5, 5, out=gradient)

    # gradient descent
    for weights, gradient in zip([Wxh, Whh, Why, bh, by], [dL_dWxh, dL_dWhh, dL_Why, dL_dbh, dL_dby]):
        weights -= lr * gradient

    # break
    
    print(f"[Loss]: {loss}")

[Loss]: 0.022469084694331867
[Loss]: 0.022346227610248466
[Loss]: 0.022223335036243613
[Loss]: 0.02210050522022857
[Loss]: 0.021977831373196637
[Loss]: 0.021855401630460847
[Loss]: 0.021733299053788294
[Loss]: 0.021611601669188976
[Loss]: 0.021490382535439174
[Loss]: 0.021369709838773905
[Loss]: 0.02124964700955284
[Loss]: 0.021130252857072866
[Loss]: 0.021011581719045
[Loss]: 0.020893683622638992
[Loss]: 0.02077660445428929
[Loss]: 0.020660386135819515
[Loss]: 0.02054506680471715
[Loss]: 0.020430680996656663
[Loss]: 0.020317259828658077
[Loss]: 0.02020483118147008
[Loss]: 0.020093419879973314
[Loss]: 0.019983047870633652
[Loss]: 0.019873734395140098
[Loss]: 0.019765496159569195
[Loss]: 0.019658347498514136
[Loss]: 0.01955230053375466
[Loss]: 0.019447365327139775
[Loss]: 0.019343550027436953
[Loss]: 0.019240861010998717
[Loss]: 0.019139303016142327
[Loss]: 0.0190388792712108
[Loss]: 0.01893959161632549
[Loss]: 0.018841440618887175
[Loss]: 0.018744425682895544
[Loss]: 0.0186485451522353

In [15]:
sample("h", 10)

'ello world'